In [1]:
from torchvision.datasets import CIFAR10
from torchvision.transforms.functional import to_tensor
import torch


data = CIFAR10(root="/mnt/ssd-1/alexm/cifar10/", download=True)
images, labels = zip(*data)

X = torch.stack(list(map(to_tensor, images))).cuda()
Y = torch.tensor(labels).cuda()

# Shuffle deterministically
rng = torch.Generator(device=X.device).manual_seed(42)
perm = torch.randperm(len(X), generator=rng, device=X.device)
X, Y = X[perm], Y[perm]

X_vec = X.view(X.shape[0], -1)
k = int(Y.max()) + 1

test_size = 1024

X_vec_train = X_vec[:-test_size]
X_vec_test = X_vec[-test_size:]

X_train, X_test = X[:-test_size], X[-test_size:]
Y_train, Y_test = Y[:-test_size], Y[-test_size:]

Files already downloaded and verified


In [2]:
# state = torch.load("/home/nora/Data/erasers.pt")
# oleace = state['oleace']
# qleace = state['qleace']

qleace = torch.load('/mnt/ssd-1/lucia/ngrams-across-time/erasers_cache/test.pth')

/tmp/ipykernel_556858/361807041.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  qleace = torch.load('/mnt/ssd-1/lucia/ngrams-across-time/erasers_cache/test.pth')


In [3]:
import torch
from torch import Tensor, optim

from mdl.probe import Probe
from mlp_mixer_pytorch import MLPMixer


class MixerProbe(Probe):
    """Multi-layer perceptron with ResNet architecture."""

    def __init__(
        self,
        num_features: int,
        num_classes: int = 2,
        device: str | torch.device = "cpu",
        dtype: torch.dtype | None = None,
    ):
        super().__init__(num_features, num_classes, device, dtype)

        self.mixer = MLPMixer(
            image_size=32,
            channels=3,
            patch_size=4,
            dim=768,
            depth=16,
            num_classes=k,
        ).to(dtype=dtype).to(device=device)

    def forward(self, x: Tensor) -> Tensor:
        return self.mixer(x).squeeze(-1)

    def build_optimizer(self) -> optim.Optimizer:
        return optim.SGD(
            self.parameters(), lr=0.005, momentum=0.9, weight_decay=5e-4,
        )


class VitProbe(Probe):
    def __init__(
        self,
        num_classes: int = 2,
        device: str | torch.device = "cpu",
        dtype: torch.dtype | None = None,
    ):
        super().__init__(3, num_classes, device, dtype)

        from vit_pytorch import ViT

        self.vit = ViT(
            channels=3,
            depth=6,
            dim=512,
            dropout=0.1,
            #emb_dropout=0.1,
            heads=8,
            image_size=32,
            mlp_dim=1024,
            num_classes=k,
            patch_size=4,
        ).to(dtype=dtype).to(device=device)

    def forward(self, x: Tensor) -> Tensor:
        return self.vit(x).squeeze(-1)

    def build_optimizer(self) -> optim.Optimizer:
        # Implicitly does learning rate warmup in a principled way
        #return optim.SGD(self.parameters(), lr=0.005, momentum=0.9, weight_decay=0.1)
        return optim.Adam(self.parameters())

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from torchvision.transforms.v2 import RandAugment, AutoAugment
import torchvision as tv

image_size = X.shape[-1]
padding = round(image_size * 0.125)

augmentor = tv.transforms.Compose(
    [
        tv.transforms.RandomCrop(image_size, padding=padding),
        tv.transforms.RandomHorizontalFlip(),
        #tv.transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]),
        # AutoAugment()
    ]
)

In [5]:
X.shape[-1]

32

In [6]:
X_vec.shape[1]

3072

In [34]:
from mdl import Sweep, MlpProbe

def debug(x):
    print(x.shape)

image_size = X.shape[-1]
padding = round(image_size * 0.125)
# mlp_augmentor = tv.transforms.Compose(
#     [
#         tv.transforms.RandomCrop(image_size, padding=padding, padding_mode='reflect'),
#         tv.transforms.RandomHorizontalFlip(),
#         tv.transforms.Lambda(debug),
#         tv.transforms.Lambda(lambda x: x.flatten(1, 3)),
#     ]
# )

sweep = Sweep(
    X_vec.shape[1], k, device=X.device, dtype=torch.bfloat16,
    num_chunks=10,
    probe_cls=MlpProbe,
)
results = [
    sweep.run(X_train.bfloat16().flatten(1, 3), Y_train, seed=i, reduce_lr_on_plateau=False) # augment=mlp_augmentor, 
    for i in range(5)
]
results_ = [
    sweep.run(
        X_train.bfloat16().flatten(1, 3), Y_train, seed=i,
        transform=lambda x, y: qleace.eraser(x.flatten(1), y).reshape_as(x),
        augment=augmentor,
    )
    for i in range(5)
]

 89%|████████▉ | 8/9 [01:41<00:20, 20.38s/scales, loss=2.1624]

In [5]:
from mdl import VisionProbe

probe = VisionProbe(k, device=X.device)
probe.fit(X_train, Y_train, reduce_lr_on_plateau=False, max_epochs=200, verbose=True, early_stop_epochs=10, augment=augmentor)

Epoch:   2%|▏         | 4/200 [00:40<32:48, 10.04s/it, loss=1.14]


KeyboardInterrupt: 

In [ ]:
probe.accuracy(X_test, Y_test, 128)

In [5]:
def to_bytes(x):
    return x.mul(255).byte()

def to_float(x):
    return x.float().div(255)


In [6]:
from torchvision.transforms.functional import to_pil_image

with torch.autocast("cuda"):
    vit = VitProbe(device=X.device)
    vit.fit(X_train, Y_train, verbose=True, reduce_lr_on_plateau=True, max_epochs=200, early_stop_epochs=50, augment=augmentor, batch_size=100)

NameError: name 'VitProbe' is not defined

In [ ]:
vit.accuracy(X_test, Y_test, 128)

In [8]:
print(X_vec.shape)

torch.Size([50000, 3072])


In [ ]:
# results_linear_ = [
#     sweep.run(
#         X_train.bfloat16(), Y_train, seed=i,
#         transform=lambda x, y: oleace.eraser(x.flatten(1), y),
#         augment=augmentor,
#     )
#     for i in range(5)
# ]

In [5]:
import pickle

with open("/home/nora/Data/cifar-results/mlp16.pkl", "wb") as f:
    pickle.dump({
        'results': results,
        'results_': results_,
        # 'results_linear_': results_linear_,
    }, f)

In [6]:
import numpy as np

curve = np.median([res.ce_curve for res in results], axis=0)
curve_ = np.median([res.ce_curve for res in results_], axis=0)
# curve_linear_ = np.median([res.ce_curve for res in results_linear_], axis=0)

In [ ]:
# sum(p.numel() for p in mixer.parameters())

In [ ]:
import matplotlib.pyplot as plt

for i in range(5):
    plt.plot(results[i].sample_sizes[:-1], results[i].ce_curve, alpha=0.15, c="black")
    plt.plot(results[i].sample_sizes[:-1], results_[i].ce_curve, alpha=0.15, c="black")
    # plt.plot(results[i].sample_sizes[:-1], results_linear_[i].ce_curve, alpha=0.15, c="black")

plt.plot(results[0].sample_sizes[:-1], curve, label="Original", marker="o")
plt.plot(results[0].sample_sizes[:-1], curve_, label="Q-LEACE", marker="o")
# plt.plot(results[0].sample_sizes[:-1], curve_linear_, label="LEACE", marker="o")

plt.hlines(np.log2(10), 0, 2 ** 16, label="Chance", linestyle="--", color="black")
plt.legend()
plt.xscale("log", base=2)
plt.xlabel("# training samples")
plt.ylabel("Cross-entropy (bits)")
plt.title("CIFAR-10 (16-Layer MLP)")
plt.yscale("log", base=2)

In [ ]:
resnet = tv.models.resnet18(pretrained=True)

In [ ]:
resnet.layer1[0]

In [ ]:
from torchvision.models.resnet import BasicBlock

In [ ]:
from torch import nn

class MlpBlock(nn.Module):
    def __init__(
        self, in_features: int, out_features: int, device = None, dtype = None
    ):
        super().__init__()

        self.linear1 = nn.Linear(
            in_features, out_features, bias=False, device=device, dtype=dtype
        )
        self.linear2 = nn.Linear(
            out_features, out_features, bias=False, device=device, dtype=dtype
        )
        self.bn1 = nn.BatchNorm1d(
            in_features, device=device, dtype=dtype
        )
        self.bn2 = nn.BatchNorm1d(
            out_features, device=device, dtype=dtype
        )
        self.downsample = nn.Linear(
            in_features, out_features, bias=False, device=device, dtype=dtype
        ) if in_features != out_features else None

    def forward(self, x):
        identity = x
    
        out = self.linear1(x)
        out = self.bn1(out)
        out = nn.functional.relu(out)

        out = self.linear2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = nn.functional.relu(out)

        return out